This is my most up to date and as annotated as possible script. For the data set, have it in you local directory (It's uploaded to github too)

In [1]:
#import dependences

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
np.random.seed(7) #why random?


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
def make_network(FILENAME,sklearn=False,keras=False):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    
    """
    Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and hot encoding is done. Other stuff too probably. Such as normalization, but I 
    didn't do that!
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    
    model : object
        MLP which can predict the outcome of NBA games
        
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    #*retrospectively that doesn't make sense, could be worth changing!
    data = read_csv(FILENAME) 
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    

    dat = []
    
    #reshape the dataset so now each colummn has roadstats and homestats concatenated into the same row, used for NN 
    
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    
    #convert list to array, now possible to array operations previously not possible
    dat = np.array(dat)   
    
    #now to find out the score of the game, and whether or not the spread was covered. 
    roadpts = dat[:,8] #column of all the points scored by road team 
    homepts = dat[:,21] #vice versa of above
    print("roadpts",roadpts)
    endspreadS = roadpts-homepts  #all the final spreads of the game
    openingspreadS = dat[:,9]  #what the predicted spread of ther game was. 
    y = []
    
    #Now I iterated over all these, and hot encoded the labels of each to see whether or not the spread was covered
    #and by what team. 
    for j in range(len(endspreadS)):  
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread <0:
            y.append(np.array([0,1,0]))  #hoem team covered
        elif openspread + endspread >0:
            y.append(np.array([1,0,0]))  #road covered
        else: 
            y.append(np.array([0,0,1]))  #push!



    y = np.array(y)  #same explanation as above
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)  #reshaping arrays,
    #since everything got out of order I have to mash it together myself. 
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    #now to construct a model 
    if sklearn: 
        model = MLPClassifier()
        model.shuffle = True
        model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
        model.fit(X_train,y_train)
        print(model.score(X_test,y_test))
    if keras:
        print("keras NN goes here")
        model=Sequential()
        model.add(Dense(22,input_dim=np.shape(X)[1],activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(50,activation='relu'))
        model.add(Dense(30,activation='relu'))
        model.add(Dense(22,activation='relu'))

        model.add(Dense(3,activation='sigmoid'))

        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        model.fit(X_train,y_train,batch_size=40,epochs=20,validation_split=.2)
        scores = model.evaluate(X_test,y_test)
        print(scores[1])

    
    
    return model
    

In [5]:
#For this spreadsheet, see github. 
nbapredictor = make_network('1517_boxscores.csv',sklearn=False,keras=True)

roadpts [ 88. 104. 129. ... 112. 101.  93.]
keras NN goes here
Train on 1532 samples, validate on 384 samples
Epoch 1/20
1532/1532 [==============================] - 1s 472us/step - loss: 0.7252 - acc: 0.6456 - val_loss: 0.6097 - val_acc: 0.7448
Epoch 2/20
1532/1532 [==============================] - 0s 45us/step - loss: 0.6243 - acc: 0.7226 - val_loss: 0.6708 - val_acc: 0.6615
Epoch 3/20
1532/1532 [==============================] - 0s 50us/step - loss: 0.6082 - acc: 0.7154 - val_loss: 0.5655 - val_acc: 0.7708
Epoch 4/20
1532/1532 [==============================] - 0s 51us/step - loss: 0.5873 - acc: 0.7461 - val_loss: 0.5493 - val_acc: 0.7448
Epoch 5/20
1532/1532 [==============================] - 0s 47us/step - loss: 0.5842 - acc: 0.7520 - val_loss: 0.5433 - val_acc: 0.7734
Epoch 6/20
1532/1532 [==============================] - 0s 48us/step - loss: 0.5743 - acc: 0.7480 - val_loss: 0.5398 - val_acc: 0.7474
Epoch 7/20
1532/1532 [==============================] - 0s 47us/step - loss: 0.

The next function I have is for turning the current nba team statistics (either over the entire season or some stretch) into an array of the same shape and information as the one used for the box scores above. 

In [6]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [7]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    return game

In [8]:
filetouse = '1718nbateamstats.csv'  #downloaded from basketball reference, and specified date 

teams, stats =make_prediction_data(filetouse)

Here is every teams index value: 
0     Golden State Warriors* 
1     Houston Rockets*       
2     New Orleans Pelicans*  
3     Toronto Raptors*       
4     Cleveland Cavaliers*   
5     Denver Nuggets         
6     Philadelphia 76ers*    
7     Minnesota Timberwolves*
8     Los Angeles Clippers   
9     Charlotte Hornets      
10    Los Angeles Lakers     
11    Oklahoma City Thunder* 
12    Washington Wizards*    
13    Brooklyn Nets          
14    Milwaukee Bucks*       
15    Portland Trail Blazers*
16    Indiana Pacers*        
17    New York Knicks        
18    Utah Jazz*             
19    Boston Celtics*        
20    Phoenix Suns           
21    Detroit Pistons        
22    Miami Heat*            
23    Orlando Magic          
24    Atlanta Hawks          
25    Chicago Bulls          
26    San Antonio Spurs*     
27    Dallas Mavericks       
28    Memphis Grizzlies      
29    Sacramento Kings       
30    League Average         
Name: Team, dtype: object


Playoff teams + Index #
----------------------
Warriors 0
-----------
Rockets 1
-----------
Pelicans 2
-----------
Raptors 3 
-----------
Cavs 4
-----------
76ers 6
-----------
Timberwolves 7
-----------
Thunder 11
-----------
Wizards 12
-----------
Bucks 14
-----------
Trail Blazers 15
-----------
Pacers 16
-----------
Jazz 18
-----------
Celtics 19
-----------
Heat 22
-----------
Spurs 26
-----------

Now or tomorrow it would be smark to analyze all the different games, a ton have already been played, could be worth investigating what my playoff record is after round 1 for a specific series


--Some things worth noting are playoff injuries, etc. like how the sixers are never really at their full strength and same goes for warriors but whatever

In [9]:
#Round2matchups
phibos = game_maker(stats.values[6],stats.values[19]) #philly at boston
bosphi = game_maker(stats.values[19],stats.values[6]) #vice versa
cletor = game_maker(stats.values[4],stats.values[3])  #cleveland at toronto
torcle = game_maker(stats.values[3],stats.values[4]) #vice versa
utahou = game_maker(stats.values[18],stats.values[1]) #minny at houston
houuta = game_maker(stats.values[1],stats.values[18]) #vice versa
nopgsw = game_maker(stats.values[2],stats.values[0])  # pelicans at gsw
gswnop = game_maker(stats.values[0],stats.values[2])

gswsac = game_maker(stats.values[29],stats.values[0])


In [10]:
#so who covers the various game 1s? 
nbapredictor.predict(np.array([cletor,nopgsw]))

array([[0.5215818 , 0.52082324, 0.06708987],
       [0.6409434 , 0.46189535, 0.06705207]], dtype=float32)

In [12]:
nbapredictor.predict(np.array([phibos]))

array([[0.8102558 , 0.3428352 , 0.03644867]], dtype=float32)

In [ ]:
0.8464921 + 0.45634875 + 0.10764076

In [ ]:
0.9274949 + 0.3562849 + 0.10068643

In [ ]:
minhou = game_maker(stats.values[7],stats.values[1]) #minny at houston
houmin = game_maker(stats.values[1],stats.values[7]) #vice versa

game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[13])

In [ ]:
nbapredictor.predict(np.array([game1,game2,game3]))